In [37]:
import ast
from math import nan

import zipfile

import pandas as pd
import numpy as np

pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('future.no_silent_downcasting', True)

def getData():

    # Attempt to read the data from './data.csv' and unzip it from './data.zip' if it doesn't exist
    try:
        open('./data.csv')
    except FileNotFoundError:
        with zipfile.ZipFile('./data.zip', 'r') as zip_ref:
            zip_ref.extractall('./')

    data = pd.read_csv('./data.csv', sep=',', nrows=20000)
    data = data.drop(columns=['shoe_id', 'run_count', 'true_count', 'cards_remaining', 'dealer_up', 'initial_hand'])

    data['dealer_final'] = data['dealer_final'].apply(lambda x: ast.literal_eval(x))
    data['player_final'] = data['player_final'].apply(lambda x: ast.literal_eval(x))
    data['player_final_value'] = data['player_final_value'].apply(lambda x: ast.literal_eval(x))

    data['actions_taken'] = data['actions_taken'] \
        .apply(lambda x: ast.literal_eval(x)) \
        .apply(lambda x: [x[0] , ['P'] + x[1]]  if len(x)==2 else x) \
        .apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2]]  if len(x)==3 else x) \
        .apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3]]  if len(x)==4 else x) \
        .apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3] , ['P'] + x[4]]  if len(x)==5 else x)

    data = data.explode(['player_final','player_final_value','actions_taken'])

    data['dealer_final_value'] = data['dealer_final_value'].replace('BJ',21)
    data['player_final_value'] = data['player_final_value'].replace('BJ',21)

    data['dealer_final_value'] = data['dealer_final_value'].astype('int64')
    data['player_final_value'] = data['player_final_value'].astype('int64')

    data.loc[(data['dealer_final_value'] < data['player_final_value']) & (data['player_final_value'] <= 21) , 'win'] = 1
    data.loc[data['dealer_final_value'] >  21 , 'win'] = 1
    data.loc[data['dealer_final_value'] == data['player_final_value'], 'win'] = 0
    data.loc[(data['dealer_final_value'] > data['player_final_value']) & (data['dealer_final_value'] <= 21) , 'win'] = -1
    data.loc[data['player_final_value'] > 21 , 'win'] = -1
    data['win'] = data['win'].astype('int64')

    data = data[data['win'] == 1] # We only want to train our model on results where the player has won
    data = data.drop(columns=['win'])

    def cards_splitter(data,cards,text):
        stopsign = cards.apply(lambda x: len(x))
        for i in range(1,max(stopsign)):
            data[f'{text}{i}'] = cards.apply(lambda x: nan if len(x)<i else x[i-1])

    cards_splitter(data,data['player_final'],'player_card_')
    cards_splitter(data,data['dealer_final'],'dealer_card_')
    cards_splitter(data,data['actions_taken'],'action_taken_')
    data = data.drop(columns=['player_final', 'dealer_final', 'actions_taken'])


    def decompose_actions(row):
        frame = row.to_frame().transpose()
        num_action_cols = sum([1 for column in frame if column.startswith('action_taken_')])
        num_player_card_cols = sum([1 for column in frame if column.startswith('player_card_')])
        data = []

        for action_index in range(1, num_action_cols + 1):
            column = f"action_taken_{action_index}"
            new_row = row.copy()

            offset = 0

            new_row['action_taken'] = row[column]

            if row[column] == 'H':
                offset = 1

            if row[column] is nan:
                break

            for new_action_index in range(action_index + 1, num_action_cols + 1):
                new_column = f"action_taken_{new_action_index}"
                new_row[new_column] = nan

            for player_card_index in range(action_index + 2 + offset, num_player_card_cols + 1):
                card_column = f'player_card_{player_card_index}'
                new_row[card_column] = nan

            # Drop all the action_taken columns
            frame = new_row.to_frame().transpose()
            for column in frame:
                if column.startswith('action_taken_'):
                    frame.drop(columns=[column], inplace=True)

            # frame.drop(columns=[column for column in frame if column], inplace=True)

            data.append(frame)

        return data

    new_data = pd.DataFrame()

    for index, row in data.iterrows():
        for new_rows in decompose_actions(row):
            new_data = pd.concat([new_data, new_rows], ignore_index=True)

    data = new_data
    data = data.fillna(0)

    return data


df = getData()
df.info()
display(df.describe())
display(df.head(10))

display(df['action_taken'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12872 entries, 0 to 12871
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dealer_final_value  12872 non-null  object
 1   player_final_value  12872 non-null  object
 2   player_card_1       12872 non-null  object
 3   player_card_2       12872 non-null  object
 4   player_card_3       12872 non-null  object
 5   player_card_4       12872 non-null  object
 6   player_card_5       12872 non-null  object
 7   dealer_card_1       12872 non-null  object
 8   dealer_card_2       12872 non-null  object
 9   dealer_card_3       12872 non-null  object
 10  dealer_card_4       12872 non-null  object
 11  dealer_card_5       12872 non-null  object
 12  dealer_card_6       12872 non-null  object
 13  action_taken        12872 non-null  object
dtypes: object(14)
memory usage: 1.4+ MB


,dealer_final_value,player_final_value,player_card_1,player_card_2,player_card_3,player_card_4,player_card_5,dealer_card_1,dealer_card_2,dealer_card_3,dealer_card_4,dealer_card_5,dealer_card_6,action_taken
count,12872,12872,12872,12872,12872,12872,12872,12872,12872,12872.0,12872,12872,12872,12872
unique,9,11,10,10,11,11,11,10,10,11.0,11,11,9,6
top,17,20,10,10,0,0,0,10,10,10.0,0,0,0,S
freq,2093,3140,3615,3956,7189,11213,12615,3409,3306,3796.0,9197,12086,12781,7330


,dealer_final_value,player_final_value,player_card_1,player_card_2,player_card_3,player_card_4,player_card_5,dealer_card_1,dealer_card_2,dealer_card_3,dealer_card_4,dealer_card_5,dealer_card_6,action_taken
0,24,21,10,11,0,0,0,10,4,10.0,0,0,0,S
1,18,21,5,5,11.0,0,0,10,8,0,0,0,0,H
2,18,21,5,5,11.0,0,0,10,8,0,0,0,0,S
3,22,13,3,10,0,0,0,6,6,10.0,0,0,0,S
4,26,15,3,2,0,0,0,6,10,10.0,0,0,0,P
5,26,15,3,2,10.0,0,0,6,10,10.0,0,0,0,H
6,26,15,3,2,10.0,0,0,6,10,10.0,0,0,0,S
7,26,12,3,4,0,0,0,6,10,10.0,0,0,0,P
8,26,12,3,4,5.0,0,0,6,10,10.0,0,0,0,H
9,26,12,3,4,5.0,0,0,6,10,10.0,0,0,0,S


action_taken
S    7330
H    3205
D    1190
P     498
N     417
R     232
Name: count, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def pickTarget(df, targets):
  X = df.copy()
  y = X[targets]

  X.drop(columns=targets, inplace=True)

  return X, y

def splitData(X, y, test_size = 0.33, validation_size = 0.5):
  X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size)
  X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=validation_size)

  data = {
    'x': {
      'test': X_test,
      'train': X_train,
      'validation': X_val,
    },
    'y': {
      'test': y_test,
      'train': y_train,
      'validation': y_val,
    }
  }

  return data['x'], data['y']

def scaleData(X, y):
    scX = StandardScaler()
    scalerX = scX.fit(X["train"])
    trainX_scaled = scalerX.transform(X["train"])
    testX_scaled = scalerX.transform(X["test"])

    data = {
        "x": {
            "test": testX_scaled,
            "train": trainX_scaled,
        },
        "y": y,
    }

    return data["x"], data["y"]

X, y = pickTarget(df, 'action_taken')
X, y = splitData(X, y)
X, y = scaleData(X, y)


# Logistic Regressor Baseline Model

In [39]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
# lr.fit(X['train'], y['train'])

accs = []

for train_index, test_index in KFold(n_splits=10).split(X=X['train'], y=y['train']):
    X_train, X_test = X['train'][train_index], X['train'][test_index]
    y_train, y_test = y['train'].iloc[train_index], y['train'].iloc[test_index]

    lr.fit(X_train, y_train)

    score = lr.score(X_test, y_test)
    accs.append(score)

display(f"Mean Accuracy: {np.mean(accs)}")


'Mean Accuracy: 0.6924802327175745'

# MLP Baseline Model

In [40]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(max_iter=1000)
model.fit(X['train'], y['train'])


MLPClassifier(max_iter=1000)

In [41]:
from sklearn import metrics

predicted_y = model.predict(X['test'])
print(metrics.classification_report(y['test'], predicted_y))
print(metrics.confusion_matrix(y['test'], predicted_y))


              precision    recall  f1-score   support

           D       0.91      0.90      0.91       200
           H       0.51      0.60      0.55       515
           N       0.67      0.65      0.66        63
           P       0.48      0.14      0.21        73
           R       0.82      0.86      0.84        36
           S       0.77      0.75      0.76      1237

    accuracy                           0.70      2124
   macro avg       0.69      0.65      0.65      2124
weighted avg       0.71      0.70      0.70      2124

[[180   4   0   9   0   7]
 [  4 307   0   0   0 204]
 [  0   0  41   0   6  16]
 [ 12   1   1  10   0  49]
 [  0   0   5   0  31   0]
 [  1 295  14   2   1 924]]
